<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук НИУ ВШЭ <br> Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 3. Построение дерева решений. Решение</center>

**Цель задания - "на пальцах", с помощью игрушечных задач классификации разобраться в том, как работают деревья решений. Напомню, само по себе дерево решений - довольно слабый алгоритм, но основанные на нем алгоритмы случайного леса и градиентного бустинга - пожалуй, лучшее, что есть на сегодняшний день. Поэтому разобраться в том, как работет дерево решений, полезно.**

**Рассмотрим 3 игрушечных задачи бинарной классификации (символ $\tau$ вместо знака вопроса обозначает неизвестные метки целевого класса тестовых объектов):**
<img src='../../img/toy_clf_task1.png'>
<img src='../../img/toy_clf_task2.png'>
<img src='../../img/toy_clf_task3.png'>

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections

# Деревья решений и энтропия

## Загрузка данных

In [2]:
# Создание датафрейма с dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
def intersect_features(train, test):
    f = list( set(train.keys()) & set(test.keys()))
    return train[f], test[f]

### Признаки

In [3]:
fn_1 = [u'Возраст', u'Доход', u'Внешность', u'Образование'] # знакомства
fn_2 = [u'Цвет', u'Тип', u'Производство', u'Повреждения'] # угон
fn_3 = [u'Пол', u'Возраст', u'Образование', u'Зарпалата'] # кредит

### Служба	знакомств

In [4]:
# обучение

df1_train = {}
df1_train[u'Возраст'] = [u'молодой', u'средний', u'пожилой', u'средний', u'молодой', u'средний', u'пожилой'] 
df1_train[u'Доход'] = [ u'высокий', u'высокий', u'высокий', u'средний', u'низкий', u'высокий', u'средний']
df1_train[u'Внешность'] = [ u'приятная', u'приятная', u'приятная', u'приятная', u'приятная', u'отталкивающая', u'приятная']
df1_train[u'Образование'] = [ u'специальное', u'высшее', u'высшее', u'высшее', u'специальное', u'высшее', u'среднее']
df1_train[u'Составит пару'] = LabelEncoder().fit_transform(['+','+','+','+','-','-','-'])

df1_train = create_df(df1_train, fn_1)
df1_train

,Составит пару,Возраст_молодой,Возраст_пожилой,Возраст_средний,Доход_высокий,Доход_низкий,Доход_средний,Внешность_отталкивающая,Внешность_приятная,Образование_высшее,Образование_специальное,Образование_среднее
0,0,1,0,0,1,0,0,0,1,0,1,0
1,0,0,0,1,1,0,0,0,1,1,0,0
2,0,0,1,0,1,0,0,0,1,1,0,0
3,0,0,0,1,0,0,1,0,1,1,0,0
4,1,1,0,0,0,1,0,0,1,0,1,0
5,1,0,0,1,1,0,0,1,0,1,0,0
6,1,0,1,0,0,0,1,0,1,0,0,1


In [5]:
# тест

df1_test = {}
df1_test[u'Возраст'] = [u'молодой', u'средний', u'пожилой'] 
df1_test[u'Доход'] = [u'средний', u'высокий', u'низкий']
df1_test[u'Внешность'] = [u'приятная', u'отталкивающая', u'приятная']
df1_test[u'Образование'] = [u'высшее', u'специальное', u'высшее']
df1_test = create_df(df1_test, fn_1)
df1_test

,Возраст_молодой,Возраст_пожилой,Возраст_средний,Доход_высокий,Доход_низкий,Доход_средний,Внешность_отталкивающая,Внешность_приятная,Образование_высшее,Образование_специальное
0,1,0,0,0,0,1,0,1,1,0
1,0,0,1,1,0,0,1,0,0,1
2,0,1,0,0,1,0,0,1,1,0


In [6]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y_1 = df1_train[u'Составит пару']
df1_train, df1_test = intersect_features(train = df1_train, test = df1_test)
df1_train

,Возраст_средний,Возраст_молодой,Доход_высокий,Образование_специальное,Внешность_приятная,Доход_низкий,Возраст_пожилой,Внешность_отталкивающая,Образование_высшее,Доход_средний
0,0,1,1,1,1,0,0,0,0,0
1,1,0,1,0,1,0,0,0,1,0
2,0,0,1,0,1,0,1,0,1,0
3,1,0,0,0,1,0,0,0,1,1
4,0,1,0,1,1,1,0,0,0,0
5,1,0,1,0,0,0,0,1,1,0
6,0,0,0,0,1,0,1,0,0,1


In [7]:
df1_test

,Возраст_средний,Возраст_молодой,Доход_высокий,Образование_специальное,Внешность_приятная,Доход_низкий,Возраст_пожилой,Внешность_отталкивающая,Образование_высшее,Доход_средний
0,0,1,0,0,1,0,0,0,1,1
1,1,0,1,1,0,0,0,1,0,0
2,0,0,0,0,1,1,1,0,1,0


### Автоугон

In [8]:
# обучение

df2_train = {}
df2_train[u'Цвет'] = [u'Красный', u'Желтый', u'Желтый', u'Красный', u'Желтый', u'Желтый', u'Красный'] 
df2_train[u'Тип'] = [u'Спортивный', u'Спортивный', u'Джип', u'Спортивный', u'Спортивный', u'Джип', u'Джип']
df2_train[u'Производство'] = [u'США', u'Япония', u'Япония', u'Япония', u'США', u'США', u'Япония']
df2_train[u'Повреждения'] = [u'нет', u'нет', u'нет', u'есть', u'есть', u'нет', u'есть']
df2_train[u'Угоняют'] = LabelEncoder().fit_transform(['+','+','+','+','-','-','-'])

df2_train = create_df(df2_train, fn_2)
df2_train

,Угоняют,Цвет_Желтый,Цвет_Красный,Тип_Джип,Тип_Спортивный,Производство_США,Производство_Япония,Повреждения_есть,Повреждения_нет
0,0,0,1,0,1,1,0,0,1
1,0,1,0,0,1,0,1,0,1
2,0,1,0,1,0,0,1,0,1
3,0,0,1,0,1,0,1,1,0
4,1,1,0,0,1,1,0,1,0
5,1,1,0,1,0,1,0,0,1
6,1,0,1,1,0,0,1,1,0


In [9]:
# тест

df2_test = {}
df2_test[u'Цвет'] = [u'Зеленый', u'Красный', u'Черный'] 
df2_test[u'Тип'] = [u'Спортивный', u'Спортивный', u'Джип']
df2_test[u'Производство'] = [u'США', u'Германия', u'Япония']
df2_test[u'Повреждения'] = [u'нет',u'нет',u'нет']

df2_test = create_df(df2_test, fn_2)
df2_test

,Цвет_Зеленый,Цвет_Красный,Цвет_Черный,Тип_Джип,Тип_Спортивный,Производство_Германия,Производство_США,Производство_Япония,Повреждения_нет
0,1,0,0,0,1,0,1,0,1
1,0,1,0,0,1,1,0,0,1
2,0,0,1,1,0,0,0,1,1


In [10]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y_2 = df2_train[u'Угоняют']
df2_train, df2_test = intersect_features(train = df2_train, test = df2_test)
df2_train

,Производство_США,Производство_Япония,Цвет_Красный,Тип_Спортивный,Тип_Джип,Повреждения_нет
0,1,0,1,1,0,1
1,0,1,0,1,0,1
2,0,1,0,0,1,1
3,0,1,1,1,0,0
4,1,0,0,1,0,0
5,1,0,0,0,1,1
6,0,1,1,0,1,0


In [11]:
df2_test

,Производство_США,Производство_Япония,Цвет_Красный,Тип_Спортивный,Тип_Джип,Повреждения_нет
0,1,0,0,1,0,1
1,0,0,1,1,0,1
2,0,1,0,0,1,1


### Выдача кредита

In [12]:
# обучение

df3_train = {}
df3_train[u'Пол'] = LabelEncoder().fit_transform([u'М',u'Ж',u'Ж',u'М',u'М',u'Ж',u'Ж'])
df3_train[u'Возраст'] = [u'молодой', u'средний', u'средний', u'пожилой', u'молодой', u'средний', u'пожилой']
df3_train[u'Образование'] = [u'высшее', u'специальное', u'высшее', u'высшее', u'высшее', u'среднее', u'специальное']
df3_train[u'Зарпалата'] = [u'высокая', u'высокая', u'средняя', u'высокая', u'низкая', u'средняя', u'средняя']
df3_train[u'Выдать кредит'] = LabelEncoder().fit_transform(['+','+','+','+','-','-','-'])

df3_train = create_df(df3_train, fn_3)
df3_train[u'Пол'] = LabelEncoder().fit_transform([u'М', u'Ж', u'Ж', u'М', u'М', u'Ж', u'Ж'])
df3_train


,Выдать кредит,Возраст_молодой,Возраст_пожилой,Возраст_средний,Образование_высшее,Образование_специальное,Образование_среднее,Зарпалата_высокая,Зарпалата_низкая,Зарпалата_средняя,Пол
0,0,1,0,0,1,0,0,1,0,0,1
1,0,0,0,1,0,1,0,1,0,0,0
2,0,0,0,1,1,0,0,0,0,1,0
3,0,0,1,0,1,0,0,1,0,0,1
4,1,1,0,0,1,0,0,0,1,0,1
5,1,0,0,1,0,0,1,0,0,1,0
6,1,0,1,0,0,1,0,0,0,1,0


In [13]:
# тест

df3_test = {}
df3_test[u'Пол'] =  LabelEncoder().fit_transform([u'Ж', u'Ж', u'М'])
df3_test[u'Возраст'] = [u'молодой', u'пожилой', u'средний']
df3_test[u'Образование'] = [u'специальное', u'высшее', u'специальное']
df3_test[u'Зарпалата'] = [u'высокая', u'средняя', u'средняя']

df3_test = create_df(df3_test, fn_3)
df3_test[u'Пол'] =  LabelEncoder().fit_transform([u'Ж', u'Ж', u'М'])
df3_test

,Возраст_молодой,Возраст_пожилой,Возраст_средний,Образование_высшее,Образование_специальное,Зарпалата_высокая,Зарпалата_средняя,Пол
0,1,0,0,0,1,1,0,0
1,0,1,0,1,0,0,1,0
2,0,0,1,0,1,0,1,1


In [14]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y_3 = df3_train[u'Выдать кредит']
df3_train, df3_test = intersect_features(train = df3_train, test = df3_test)
df3_train

,Возраст_средний,Пол,Возраст_молодой,Образование_специальное,Зарпалата_высокая,Возраст_пожилой,Образование_высшее,Зарпалата_средняя
0,0,1,1,0,1,0,1,0
1,1,0,0,1,1,0,0,0
2,1,0,0,0,0,0,1,1
3,0,1,0,0,1,1,1,0
4,0,1,1,0,0,0,1,0
5,1,0,0,0,0,0,0,1
6,0,0,0,1,0,1,0,1


In [15]:
df3_test

,Возраст_средний,Пол,Возраст_молодой,Образование_специальное,Зарпалата_высокая,Возраст_пожилой,Образование_высшее,Зарпалата_средняя
0,0,0,1,1,1,0,0,0
1,0,0,0,0,0,1,1,1
2,1,1,0,1,0,0,0,1


## Функции для расчета энтропии и прироста информации

In [16]:
# данные для теста: 9 синих шариков и 11 желтых
b = [1 for i in range(9)] + [0 for i in range(11)]

# два разбиения
b1 = [1 for i in range(8)] + [0 for i in range(5)] # 8 синих и 5 желтых
b2 = [1 for i in range(1)] + [0 for i in range(6)] # 1 синий и 6 желтых

In [17]:
# расчет энтропии Шеннона
def entropy(y):
    c = collections.Counter(y)
    p = map(lambda x: float(x) / len(y), c.values())
    return sum(map(lambda x: -x*np.log2(x), p))

In [18]:
print(entropy([1,0,0,0,0,0,0])) # 1 синий и 6 желтых
print(entropy(b))
print(entropy([1,2,3,4,5,6])) # энтропия игральной кости с несмещенным центром тяжести

0.591672778582
0.992774453988
2.58496250072


In [19]:
# расчет прироста информации

def information_gain(root, left, right):
    ''' root - изначальный набор данных, left и right два разбиения изначального набора'''
    N = float(len(root))
    N1 = float(len(left))
    N2 = float(len(right))
    return entropy(root) - (N1/N) * entropy(left) - (N2/N) * entropy(right)

In [20]:
print(information_gain(b,b1,b2))

0.160885188414


In [21]:
# функция для расчета прироста информации при разбиении набора данных 

def split(X, y):
    ''' Выводит прирост информации при разбиении по каждому признаку'''
    inf_gain = {}
    if isinstance(X, pd.core.frame.DataFrame):
        for k in X.columns:
            inf_gain[k] = information_gain(y, y[X[k] == 0], y[X[k] == 1])
        print(pd.DataFrame(inf_gain, index = ['IG']).T.sort(['IG']))
        
    elif isinstance(X, pd.core.series.Series):
        print(information_gain(y, y[X == 0], y[X == 1]))

## Анализ данных с помощью энтропийного критерия

Условимся, что левое поддерово то, для которого значение признака = 0

### Служба знакомств

1) Разбиение исходного набора. Видно, что максимальный прирост 0.198 достигается при разбиении на трех признаках:
 * Внешность_отталкивающая  
 * Внешность_приятная       
 * Доход_низкий

In [22]:
split(df1_train, y_1)

                               IG
Возраст_молодой          0.005978
Возраст_пожилой          0.005978
Доход_средний            0.005978
Образование_специальное  0.005978
Возраст_средний          0.020244
Доход_высокий            0.128085
Образование_высшее       0.128085
Внешность_отталкивающая  0.198117
Внешность_приятная       0.198117
Доход_низкий             0.198117


 2) Выберем для следующего разбиения признак "Внешность_приятная". Левое поддерево:

In [23]:
df1_train

,Возраст_средний,Возраст_молодой,Доход_высокий,Образование_специальное,Внешность_приятная,Доход_низкий,Возраст_пожилой,Внешность_отталкивающая,Образование_высшее,Доход_средний
0,0,1,1,1,1,0,0,0,0,0
1,1,0,1,0,1,0,0,0,1,0
2,0,0,1,0,1,0,1,0,1,0
3,1,0,0,0,1,0,0,0,1,1
4,0,1,0,1,1,1,0,0,0,0
5,1,0,1,0,0,0,0,1,1,0
6,0,0,0,0,1,0,1,0,0,1


In [24]:
df1_train_left = df1_train[df1_train[u'Внешность_приятная'] == 0]
df1_train_right = df1_train[df1_train[u'Внешность_приятная'] == 1]

In [25]:
split(df1_train_left, y_1[df1_train_left.index])

                          IG
Внешность_отталкивающая  0.0
Внешность_приятная       0.0
Возраст_молодой          0.0
Возраст_пожилой          0.0
Возраст_средний          0.0
Доход_высокий            0.0
Доход_низкий             0.0
Доход_средний            0.0
Образование_высшее       0.0
Образование_специальное  0.0


Правое поддерево:

In [26]:
split(df1_train_right, y_1[df1_train_right.index])

                               IG
Внешность_отталкивающая  0.000000
Внешность_приятная       0.000000
Возраст_молодой          0.044110
Возраст_пожилой          0.044110
Доход_средний            0.044110
Образование_специальное  0.044110
Возраст_средний          0.251629
Доход_низкий             0.316689
Доход_высокий            0.459148
Образование_высшее       0.459148


В левом поддереве получился прирост 0, поскольку там оказался один объект. В правом поддереве сделаем разбиение по признаку "Образование_высшее".

Предполагается, что новый объект относится к тому классу, объектов из обучающей выборки которого в листе большинство

<img src='../../img/toy_task_tree1_by_hand.png' width=50%>

Согласно построенному дереву, тестовые объекты следует классифицировать так:
1. Да
2. Нет
3. Да

### Автоугон

Аналогичные рассуждения:

In [27]:
split(df2_train, y_2)

                           IG
Цвет_Красный         0.020244
Повреждения_нет      0.128085
Производство_США     0.128085
Производство_Япония  0.128085
Тип_Джип             0.128085
Тип_Спортивный       0.128085


In [28]:
split(df2_train[df2_train[u'Тип_Спортивный'] == 0], y_2[df2_train[u'Тип_Спортивный'] == 0])

                           IG
Тип_Джип             0.000000
Тип_Спортивный       0.000000
Повреждения_нет      0.251629
Производство_США     0.251629
Производство_Япония  0.251629
Цвет_Красный         0.251629


In [29]:
split(df2_train[df2_train[u'Тип_Спортивный'] == 1],y_2[df2_train[u'Тип_Спортивный'] == 1])

                           IG
Тип_Джип             0.000000
Тип_Спортивный       0.000000
Повреждения_нет      0.311278
Производство_США     0.311278
Производство_Япония  0.311278
Цвет_Красный         0.311278


In [30]:
y_2[(df2_train[u'Тип_Спортивный'] == 0) & (df2_train[u'Цвет_Красный'] == 0)] 

2    0
5    1
Name: Угоняют, dtype: int64

In [31]:
y_2[(df2_train[u'Тип_Спортивный'] == 0) & (df2_train[u'Цвет_Красный'] == 1)]

6    1
Name: Угоняют, dtype: int64

In [32]:
y_2[(df2_train[u'Тип_Спортивный'] == 1) & (df2_train[u'Цвет_Красный'] == 0)]

1    0
4    1
Name: Угоняют, dtype: int64

In [33]:
y_2[(df2_train[u'Тип_Спортивный'] == 1) & (df2_train[u'Цвет_Красный'] == 1)]

0    0
3    0
Name: Угоняют, dtype: int64

<img src='../../img/toy_task_tree2_by_hand.png' width=50%>

Согласно построенному дереву, тестовые объекты следует классифицировать так:
1. Нет
2. Да
3. Нет

### Выдача кредита

In [34]:
split(df3_train, y_3)

                               IG
Возраст_молодой          0.005978
Возраст_пожилой          0.005978
Образование_специальное  0.005978
Возраст_средний          0.020244
Пол                      0.020244
Зарпалата_средняя        0.128085
Образование_высшее       0.128085
Зарпалата_высокая        0.521641


In [35]:
split(df3_train[df3_train[u'Зарпалата_высокая'] == 0], y_3[df3_train[u'Зарпалата_высокая'] == 0])

                               IG
Зарпалата_высокая        0.000000
Возраст_молодой          0.122556
Возраст_пожилой          0.122556
Зарпалата_средняя        0.122556
Образование_специальное  0.122556
Пол                      0.122556
Возраст_средний          0.311278
Образование_высшее       0.311278


In [36]:
split(df3_train[df3_train[u'Зарпалата_высокая'] == 1], y_3[df3_train[u'Зарпалата_высокая'] == 1])

                          IG
Возраст_молодой          0.0
Возраст_пожилой          0.0
Возраст_средний          0.0
Зарпалата_высокая        0.0
Зарпалата_средняя        0.0
Образование_высшее       0.0
Образование_специальное  0.0
Пол                      0.0


In [37]:
y_3[df3_train[u'Зарпалата_высокая'] == 1]

0    0
1    0
3    0
Name: Выдать кредит, dtype: int64

In [38]:
y_3[(df3_train[u'Зарпалата_высокая'] == 0) & (df3_train[u'Образование_высшее'] == 0)]

5    1
6    1
Name: Выдать кредит, dtype: int64

In [39]:
y_3[(df3_train[u'Зарпалата_высокая'] == 0) & (df3_train[u'Образование_высшее'] == 1)]

2    0
4    1
Name: Выдать кредит, dtype: int64

<img src='../../img/toy_task_tree3_by_hand.png' width=50%>

Согласно построенному дереву, тестовые объекты следует классифицировать так:
1. Да
2. Да
3. Нет